# Mervar - Kohonen Maps
### Alexander Mervar
### Professor Brown
### COGS-Q 355 SP22
### 2.9.2022

Consider a Kohonen self organizing feature map (SOFM), as described in Kohonen (1982). Consider a map with 2500 elements arranged in a 50x50 rectangular grid (matrix). Each input vector has 3 elements in the continuous range of [0,1] that correspond to a signal of one of the primary colors (red, green, blue). For example, (1, 0, 0) is an input pattern that corresponds to the color red.

Implement a simple SOFM in python following the equations described in Kohonen (1982), but simplified as follows. Assume a weight matrix W of dimensions 50x50x3, that connects each input to each SOFM unit. The input to each SOFM unit is the external input plus the lateral input:
- External Input = W * inputs (Equation 4 in Kohonen (1982))
- Activity from A (from perspective of B):
  - a * 8 ; If euclidean distance is between A and B < 3
  - a * -1 ; if euclidean distance is between A and B is > 3 < 8
  - 0 otherwise
- The total activity $\eta$ ("eta") of each SOFM is a piece-wise sigmoid
  - 0 if the sum of the external and lateral input is <= 0
  - 5 if the sum of the external and lateral input is >= 5
  - The sum of the external and lateral input, otherwise
- Weights W from the external inputs to the SOFM units are adjusted by the following rule
  - W(t+1) = (W(t) + learning*eta*inputs)/(the euclidean length of the input weights to the SOFM unit)
  - learning = 0.01

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from math import dist

learningRate = 0.01

# Define an random 50x50x3 array
weightMatrix = np.random.random((50, 50, 3))

# Traverse the 5x50x3 matrix
for i in range(50):
    #
    for j in range(50):
        #
        for k in range(3):
            # 

In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from math import dist

class SOFM():

    learningRate = 0.01

    # Define a self-organizing feature map of size x*y with each SOFM unit having n weights
    def __init__(self, x, y, numWeights):
        self.x = x
        self.y = y
        self.weights = numWeights
        map = np.zeros((numWeights, x, y))

    # Initialize the weights of the SOFM with 3 rations
    # Note: The wights are cumulative, i.e. the third weight is the sum of the first two weights
    # So a map of 1/3 red and 2/3 green is initialized as follows: [0.33, 1.0, 1.0]
    # A map of 1/3 red and 2/3 blue is initialized as follows: [0.33, 0.33 1.0]
    def initializeWeights(self, red, green, blue):
        # traverse each of the 3 weights attached to each particular SOFM unit
        for i in range(self.x):
            for j in range(self.y):
                # Assign a random value between 0 and 1
                random = random.random()
                # Assign a value to one of the three weights depending on the ratio
                if random < red:
                    self.map[0][i][j] = 1
                    self.map[1][i][j] = 0
                    self.map[2][i][j] = 0
                elif random < green:
                    self.map[0][i][j] = 0
                    self.map[1][i][j] = 1
                    self.map[2][i][j] = 0
                elif random < blue:
                    self.map[0][i][j] = 0
                    self.map[1][i][j] = 0
                    self.map[2][i][j] = 1

    # Get the euclidean distance between two points
    def getDistance(self, x1, y1, x2, y2):
        point1 = (x1, y1)
        point2 = (x2, y2)
        return dist(point1, point2)

    # TODO
    # Get internal input
    def getInternalInput(self, x, y):
        return self.map[0][x][y] + self.map[1][x][y] + self.map[2][x][y]

    # TODO
    # Get external input
    def getLateralInput(self, x, y):
        # return a 1x3 vector of the weights of the SOFM unit

    #TODO
    # Calculate the eta of the SOFM
    def getEta(self, x1, y1, x2, y2):
        if (getInternalInput(x1, y1) + getLateralInput(x1, y1, x2, y2) <= 0):
            return 0
        elif (getInternalInput(x1, y1) + getLateralInput(x1, y1, x2, y2) >= 5):
            return 5
        else:
            return getInternalInput(x1, y1) + getLateralInput(x1, y1, x2, y2)
    
    # Get the RGB color of the particular SOFM unit
    def getColor(self, x, y):
        return (self.map[0][x][y] * 255, self.map[1][x][y] * 255, self.map[2][x][y] * 255)

    
    # Update weights
    def updateWeights(self, x, y, w1, w2, w3, x2, y2):
        # Update the weights of the SOFM unit
        self.map[0][x][y] = (self.map[0][x][y] + self.learningRate * getEta(x,y,x2,y2) * w1)/self.getDistance(x, y, x2, y2)
        self.map[1][x][y] = (self.map[1][x][y] + self.learningRate * getEta(x,y,x2,y2) * w2)/self.getDistance(x, y, x2, y2)
        self.map[2][x][y] = (self.map[2][x][y] + self.learningRate * getEta(x,y,x2,y2) * w3)/self.getDistance(x, y, x2, y2)

    # View the SOFM map
    def viewMap(self):
        image = np.zeros((self.x, self.y))
        for i in range(self.x):
            for j in range(self.y):
                image[i][j] = self.getColor(i, j)
        plt.imshow(image)

IndentationError: expected an indented block (<ipython-input-2-aa82ee631ac5>, line 58)